In [ ]:
import pickle
import torch
from datetime import datetime
from recbole.config import Config
from recbole.evaluator.evaluator import Evaluator

import pandas as pd
import numpy as np
import time

import os

list_dataset = [
    "Amazon_Luxury_Beauty",
    "lastfm",
    "ml-1m",
    "book-crossing",
    "Amazon_Industrial_and_Scientific",
    "Amazon_Digital_Music"
]

model_list = [
    "BPR",

    ]

window = 5
total_k = 10

results = {}

filenames_of_best_model = [
 'filename_best_model_for_Amazon_Luxury_Beauty.pickle',
 'filename_best_model_for_lastfm.pickle',
 'filename_best_model_for_ml-1m.pickle',
  'filename_best_model_for_book-crossing.pickle',
 'filename_best_model_for_Amazon_Industrial_and_Scientific.pickle',
'filename_best_model_for_Amazon_Digital_Music.pickle',
 ]

for dataset in list_dataset:
    results[dataset] = dict()

    config = Config(
                model="Pop", 
                dataset=dataset, 
                config_file_list=["../RecBole/recbole/properties/overall.yaml"],
                config_dict={"topk":5,
                 "metrics":["RelMetrics","FairWORel","IID_AID"]
                }
                )


    with open(f"../train_val_test/{dataset}_test.pickle","rb") as f:
        data = pickle.load(f)
    test = pd.DataFrame(data)

    item_id = config.final_config_dict["ITEM_ID_FIELD"]

    test = test.groupby("user_id")\
        .agg(lambda x: [x for x in x])\
        [item_id]
    evaluator = Evaluator(config)

    filename = [f for f in filenames_of_best_model if dataset in f][0]

    with open("../results/"+filename,"rb") as f:
        models = pickle.load(f)

    #non-pop model
    for model in models:

        model_name = model.split("-")[0]
        print(model_name)
        if model_name not in model_list:
            continue

        results[dataset][model_name] = dict()
        with open(f"../struct/struct_{dataset}_"+model,"rb") as f:
            struct = pickle.load(f)

        item_matrix = struct.get('rec.items')
        rec_mat = struct.get('rec.topk')
        struct.set("data.pos_items",test.apply(np.array).values)


        for k in range(0,5,1):
            print(f"Doing {dataset}, {model_name}, {k}:{k+window}")

            #slice item_matrix and relevance_matrix
            updated_item_matrix = item_matrix[:,k:k+window]
            updated_rec_mat = torch.cat((rec_mat[:,k:k+window], rec_mat[:,-1:]), axis=1)

            struct.set("rec.items",updated_item_matrix)
            struct.set("rec.topk",updated_rec_mat)
            start_time = time.time()
            evaluator = Evaluator(config)
            result = evaluator.evaluate(struct)
            print("total time taken: ", time.time() - start_time)

            results[dataset][model_name][f"{str(k+1)}-{str(k+window)}"] = result
        with open(f'sliding_result_{dataset}_{model_name}.pickle', 'wb') as f:
              pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)